<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from pathlib import Path

pd.options.mode.chained_assignment = None  # default='warn'

In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/Academia/01_Publications/00_Alex/005_RNC2')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [4]:
edgelist.columns

Index(['source', 'target', 'distance', 'bearing', 'IgnProb_bl'], dtype='object')

In [0]:
# %%timeit
def wind_scenario(wind_data):
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(edges=edgelist):
    rng = np.random.uniform(0, 1, size=edges.values.shape[0])
    mask = rng < edges.IgnProb_bl.values
    NewActiveEdges = edges[mask]
    return NewActiveEdges


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


**Main**


---



In [9]:
%%time
n=100
listScenarioDataframes = []
for scenario in range(n):
    # initial setup
    condition = True
    # listScenarioDataframes = []
    listActivatedSources = []
    time = 0 
    # wind conditions
    w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
    # ignition / initial state and edges selection
    ActiveEdges = ignition()
    # print("number of Ignitions: {}, first 5 sources: {}, first 5 targets: {}". format(len(ActiveEdges), ActiveEdges.source.head().values, ActiveEdges.target.head().values))
    if ActiveEdges.empty:
        continue
    while condition:
        # print("##### scenario : {} time : {} ######".format(scenario, time))
        # print("ActiveEdges before mask: {}, first 5 sources: {}, first 5 targets: {}". format(len(ActiveEdges), ActiveEdges.source.head().values, ActiveEdges.target.head().values))
        ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
        # print("ActiveEdges after mask: {}, first 5 sources: {}, first 5 targets: {}". format(len(ActiveEdges), ActiveEdges.source.head().values, ActiveEdges.target.head().values))
        if ActiveEdges.empty:
            break
        listScenarioDataframes.append(ActiveEdges)
        listActivatedSources.extend(ActiveEdges.source.values)
        # print(ActiveEdges)
        ActiveEdges = propagation(ActiveEdges)
        # print("ActiveEdges after propagation: {}, first 5 sources: {}, first 5 targets: {}". format(len(ActiveEdges), ActiveEdges.source.head().values, ActiveEdges.target.head().values))
        # print(ActiveEdges)
        # advance time
        time += 1
    print("##### scenario : {} time : {} ######".format(scenario, time))
# if while loop broken, save activation for the scenario
Activations = pd.concat(listScenarioDataframes)
Activations.to_parquet(folder / 'output' / 'scenario{}_Activations.parquet'.format(scenario), engine='pyarrow')

##### scenario : 0 time : 21 ######
##### scenario : 1 time : 47 ######
##### scenario : 2 time : 41 ######
##### scenario : 3 time : 29 ######
##### scenario : 4 time : 55 ######
##### scenario : 5 time : 1 ######
##### scenario : 6 time : 45 ######
##### scenario : 7 time : 43 ######
##### scenario : 8 time : 45 ######
##### scenario : 9 time : 31 ######
##### scenario : 10 time : 26 ######
##### scenario : 11 time : 49 ######
##### scenario : 12 time : 1 ######
##### scenario : 13 time : 13 ######
##### scenario : 14 time : 33 ######
##### scenario : 15 time : 18 ######
##### scenario : 16 time : 24 ######
##### scenario : 17 time : 21 ######
##### scenario : 18 time : 11 ######
##### scenario : 19 time : 14 ######
##### scenario : 20 time : 40 ######
##### scenario : 21 time : 13 ######
##### scenario : 22 time : 12 ######
##### scenario : 23 time : 33 ######
##### scenario : 24 time : 14 ######
##### scenario : 25 time : 43 ######
##### scenario : 26 time : 14 ######
##### scenari